# Entradas y Salidas de bicicletas a estaciones

Se pretende visualizar como es el comportamiento de las estaciones según la cantidad de bicicletas que se enganchan y desenganchan en cada una, teniendo en cuenta día de la semana y horario.

In [1]:
import json
import pandas as pd
from PIL import Image, ImageDraw
import geopandas as gpd
from shapely.geometry import Point
import geopandas as gpd
import folium
import psycopg2
conn = psycopg2.connect("host='postgre-sqltest.cpdeokpzufj1.us-west-2.rds.amazonaws.com' port=5432 dbname='postgres' user=xseed password=LosTilos114")
crs = {'init': 'epsg:4326'}
import numpy as np
import scipy.ndimage.filters
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import funciones_movimientos as fm #Funciones customizadas

Consulta sobre la base de datos, NO teniendo en cuenta los movimientos realizados por los trabajadores de Bicimad. Esos se analizarán aparte.

In [2]:
fecha_origen = '2018-03-21 08:00:00'
fecha_destino = '2018-06-21 23:00:00'
user_type_codes = '(0,1,2)' #Todos menos los trabajadores
data = fm.get_data_for_trip_counts(fecha_origen, fecha_destino,user_type_codes)

En este primer ejemplo se quiere visualizar como es la migración de las bicicletas, si hay alguna relación entre la hora del día y las salidas o arribos de las bicis a las estaciones

Asignamos un color diferente según el balance entre salidas y arribos a la estación en esa hora. Si hay mas salidas que arribos el color es naranja, de lo contrario se utiliza azul. El radio depende del número del balance, cuanto mayor el número mayor el radio.

In [3]:
# Ver comportamiento entre las 7 y 9 AM dias laborales
trip_counts = fm.get_trip_counts_by_hour(7,9, data, weekends = 0)
fm.plot_station_counts(trip_counts, data, radius_divisor = 50)

In [5]:
#Tabla vinculada al mapa anterior
tmp = fm.get_table_counts_by_hour(trip_counts, max_rows = 200, orderAsc = 0)
tmp

,Estacion,Barrio,Llegadas,Salidas,Balance
55,Plaza de Lavapiés,Embajadores,440,1588,1148
75,Menéndez Pelayo,Jerónimos,549,1696,1147
125,Santa María de la Cabeza,Palos de Moguer,547,1667,1120
41,Plaza de la Cebada,Palacio,506,1561,1055
40,Plaza de los Carros,Palacio,208,1240,1032
79,Puerta de Granada,Pacífico,611,1641,1030
45,Jesús y María,Embajadores,47,1010,963
171,Segovia 45,Imperial,821,1775,954
44,Santa Isabel,Embajadores,334,1281,947
164,Fernando el Católico,Arapiles,638,1580,942


In [ ]:
# Ver comportamiento de 18 a 20 PM
trip_counts = fm.get_trip_counts_by_hour(18,20, data, weekends = 0)
fm.plot_station_counts(trip_counts, data, radius_divisor = 20)

In [ ]:
#Tabla vinculada al mapa anterior
fm.get_table_counts_by_hour(trip_counts, max_rows = 200, orderAsc = 1)

Se puede visualizar como hay regiones que varian su balance entradas/salidas según la hora del día. Muchas de las que tienen mayor cantidad de partidas durante la mañana son de las que tienen más arribos al finalizar el día. Las del centro (que son las estaciones más utilizadas) mantienen una demanda constante en cualquier horario del día.

# Fines de semana

In [12]:
trip_counts = fm.get_trip_counts_by_hour(7,9, data, weekends = 1)
fm.plot_station_counts(trip_counts, data, radius_divisor = 20)

# Comportamiento de los trabajadores de bicimad

Queremos visualizar cuales son las estaciones a la que mas bicicletas aportan los trabajadores y de donde obtienen más para llevar a otras.

In [6]:
fecha_origen = '2018-03-21 08:00:00'
fecha_destino = '2018-06-21 23:00:00'
user_type_codes = '(3)'
data_trabajadores = fm.get_data_for_trip_counts(fecha_origen, fecha_destino,user_type_codes)

In [7]:
#De 7 a 9 AM
trip_counts = fm.get_trip_counts_by_hour(7,9, data_trabajadores, weekends = 0)
fm.plot_station_counts(trip_counts, data_trabajadores, radius_divisor = 30)

In [9]:
#Tabla vinculada al mapa anterior
fm.get_table_counts_by_hour(trip_counts, max_rows = 200, orderAsc = 1)

,Estacion,Barrio,Llegadas,Salidas,Balance
73,Menéndez Pelayo,Jerónimos,560.0,14,-546.0
77,Puerta de Granada,Pacífico,523.0,23,-500.0
121,Santa María de la Cabeza,Palos de Moguer,464.0,27,-437.0
39,Plaza de la Cebada,Palacio,364.0,6,-358.0
53,Plaza de Lavapiés,Embajadores,349.0,19,-330.0
38,Plaza de los Carros,Palacio,326.0,2,-324.0
166,Segovia 45,Imperial,296.0,16,-280.0
159,Fernando el Católico,Arapiles,329.0,76,-253.0
72,Puerta del Pacífico,Jerónimos,236.0,6,-230.0
154,General Álvarez de Castro,Trafalgar,312.0,91,-221.0


In [ ]:
#Ver comportamiento trabajadores de 18 a 20
trip_counts = fm.get_trip_counts_by_hour(18,20, data_trabajadores, weekends = 0)
fm.plot_station_counts(trip_counts, data_trabajadores, radius_divisor = 10)

# Visualización animada según hora del día

In [ ]:


    locations = data.groupby(["hour","code_station_departure"]).first()
    # and select only the tree columns we are interested in
    locations = locations.loc[:, ["latitude_departure",
                              "longitude_departure",
                              "name_departure"]]
    
    subset = data
    subset_arrival = data
    
    departure_counts =  subset.groupby(["hour","code_station_departure"]).count()
    # select one column
    departure_counts = departure_counts.iloc[:,[0]]
    # and rename that column
    departure_counts.columns= ["Departure Count"]

    arrival_counts =  subset_arrival.groupby(["hour","code_station_arrival"]).count()
    # select one column
    arrival_counts = arrival_counts.iloc[:,[0]]
    # and rename that column
    arrival_counts.columns= ["Arrival Count"]
    trip_counts_by_hour = departure_counts.join(locations, on=["hour","code_station_departure"]).join(arrival_counts, on = ["hour","code_station_departure"])
    
    trip_counts_by_hour = trip_counts_by_hour.reset_index()
    trip_counts_by_hour


In [ ]:
import os
os.environ["PATH"] += os.pathsep + "."

In [ ]:
def interpolate(df1, df2, x):
    """return a weighted average of two dataframes"""
    df = df1 * (1 - x) + df2 * x
    return df.replace(np.nan, 0)
  

def get_trip_counts_by_minute(float_hour):
    """get an interpolated dataframe for any time, based
    on hourly data"""
    
    columns = ["latitude_departure",
               "longitude_departure",
               "Departure Count",
               "Arrival Count"]
    df1 = get_trip_counts_by_hour(int(float_hour))
    df2 = get_trip_counts_by_hour(int(float_hour) + 1)
    
    df = interpolate(df1.loc[:,columns], 
                     df2.loc[:,columns], 
                     float_hour % 1)
    
    df["name_departure"] = df1["name_departure"]
    return df

In [ ]:
data2 = get_trip_counts_by_minute(9.5)
plot_station_counts(data2)

In [ ]:
def go_arrivals_frame(i, hour_of_day, save_path):
    
    # create the map object
    data = get_trip_counts_by_minute(hour_of_day)
    my_frame = plot_station_counts(data)
    
    # generate the png file as a byte array
    png = my_frame._to_png()
    
    #  now add a caption to the image to indicate the time-of-day.
    hour = int(hour_of_day)
    minutes = int((hour_of_day % 1) * 60)
    
    # create a PIL image object
    image = Image.open(io.BytesIO(png))
    draw = ImageDraw.ImageDraw(image)
    
    # load a font
    font = ImageFont.truetype("Roboto-Light.ttf", 30)
    
    # draw time of day text
    draw.text((20,image.height - 50), 
              "time: {:0>2}:{:0>2}h".format(hour, minutes),
              fill=(255, 255, 255), 
              font=font)
    
    # draw title
    draw.text((image.width - 400,20), 
              "Net Arrivals vs Time of Day",
              fill=(255, 255, 255), 
              font=font)
    
    # write to a png file
    filename = os.path.join(save_path, "frame_{:0>5}.png".format(i))
    image.save(filename, "PNG")
    return image

In [ ]:
arrival_times = np.arange(6, 23, .2)
frames_to_redo = [27, 41, 74, 100, 105]
for i in frames_to_redo:
    hour = arrival_times[i]
    go_arrivals_frame(i, hour, "frames")

# Relaciones entre estaciones

Se pretende visualizar ahora como se relacionan las estaciones según la hora del día. O sea como es la relación origen - destino en los movimientos de las bicicletas.

Veo en tabla viajes entre distritos por hora.

In [ ]:
fecha_origen = '2018-06-21 08:00:00'
fecha_destino = '2018-09-21 23:00:00'
user_type_codes = '(0,1,2)' #Todos menos los trabajadores
data = fm.get_data_for_trip_counts(fecha_origen, fecha_destino,user_type_codes)

In [ ]:
tmp = data.groupby(['hour','distrito_departure', 'distrito_arrival']).count().reset_index().loc[:, ["hour", "distrito_departure", "distrito_arrival", "user_type_code"]]
tmp.columns = ['hora', 'distrito origen', 'distrito destino', 'Cantidad movimientos']
tmp[tmp.hora == 7].sort_values(by=["Cantidad movimientos"], ascending=False).head(50)

Visualizo tabla de movimientos por estacion

In [ ]:
tmp = data.groupby(['hour','name_departure', 'name_arrival']).count().reset_index().loc[:, ["hour", "name_departure", "name_arrival", "user_type_code"]]
tmp.columns = ['hora', 'estacion origen', 'estacion destino', 'Cantidad movimientos']
tmp[tmp.hora == 7].sort_values(by=["Cantidad movimientos"], ascending=False).head(50)